# MobileNet

**Load pretrained mobilenet model on Tiny Imagenet dataset**



## Pretrain Model

### Get pretrain model

In [ ]:
import torch
import torch.nn as nn


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

## Create a moblienet object

In [ ]:
mobilenet_model1 = Net()
mobilenet_model2 = Net()
mobilenet_model3 = Net()

## Load model by params

There are two ways to Load model by params.

### Method 1
**First, transform the model type to nn.DataParallel and load params**

In [ ]:
# transform the model to DataParallel
mobilenet_model1 = torch.nn.DataParallel(mobilenet_model1).cuda()

# load params into a variable
params = torch.load('mobilenet_params.pth.tar')['state_dict']

# load params to model
mobilenet_model1.load_state_dict(params)

### Method 2
** Remove the model prefix in paramters, then load the params **

Reference: 

https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/4?u=qiaoyang_luo

In [ ]:
# original saved file with DataParallel
state_dict = torch.load('mobilenet_params.pth.tar')

# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

# load params
mobilenet_model2.load_state_dict(new_state_dict)

## Load the whole model

In [ ]:
mobilenet_model3 = torch.load('mobilenet_model.pth.tar')

## Validate

**Validate model without trainning**